In [1]:
import tensorflow as tf
print(tf.__version__)

physical_device = tf.config.experimental.list_physical_devices("GPU")
if len(physical_device) > 0:
    tf.config.experimental.set_memory_growth(physical_device[0], True)

2.2.0


In [2]:
from sklearn.datasets import load_diabetes

diabetes_dataset = load_diabetes()

In [3]:
from sklearn.model_selection import train_test_split

data = diabetes_dataset['data']
targets = diabetes_dataset['target']

train_data, test_data, train_targets, test_targets = train_test_split(data, targets,
                                                                      test_size=0.1)

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = tf.keras.Sequential([
    Dense(128, activation='relu', input_shape=(train_data.shape[1],)),
    Dense(64,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)        
])

In [5]:
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [6]:
class LossAndMetricCallback(tf.keras.callbacks.Callback):
    
    def on_train_batch_end(self, batch, logs=None):
        if batch % 2 == 0:
            print("\nAfter batch {}, the loss if {:7.2f}".format(batch, logs['loss']))
            
    def on_test_batch_end(self, batch, logs=None):
        print("\nAfter batch {}, the loss is {:7.2f}.".format(batch, logs['loss']))
        
    def on_epoch_end(self, epoch, logs=None):
        print("Epoch {}: Average loss is {:7.2f}, mean absolute error is {:7.2f}"
              .format(epoch, logs['loss'], logs['mae']))
        
    def on_predict_batch_end(self, batch, logs=None):
        print("Finished prediction on batch {}!".format(batch))

In [7]:
history = model.fit(train_data, train_targets, epochs=20, batch_size=100,
                    callbacks=[LossAndMetricCallback()], verbose=False)


After batch 0, the loss if 27110.17

After batch 2, the loss if 28529.85
Epoch 0: Average loss is 28857.97, mean absolute error is  151.06

After batch 0, the loss if 29333.53

After batch 2, the loss if 28591.56
Epoch 1: Average loss is 28702.55, mean absolute error is  150.58

After batch 0, the loss if 25302.54

After batch 2, the loss if 26387.81
Epoch 2: Average loss is 28482.02, mean absolute error is  149.89

After batch 0, the loss if 29307.05

After batch 2, the loss if 28501.51
Epoch 3: Average loss is 28171.96, mean absolute error is  148.90

After batch 0, the loss if 31951.68

After batch 2, the loss if 28458.75
Epoch 4: Average loss is 27752.76, mean absolute error is  147.53

After batch 0, the loss if 28611.94

After batch 2, the loss if 27357.81
Epoch 5: Average loss is 27188.08, mean absolute error is  145.67

After batch 0, the loss if 30828.41

After batch 2, the loss if 26878.98
Epoch 6: Average loss is 26475.77, mean absolute error is  143.27

After batch 0, the 

In [8]:
model_eval = model.evaluate(test_data, test_targets, batch_size=10, 
                            callbacks=[LossAndMetricCallback()], verbose=False)


After batch 0, the loss is 15574.69.

After batch 1, the loss is 15942.19.

After batch 2, the loss is 17137.45.

After batch 3, the loss is 17275.53.

After batch 4, the loss is 18912.10.


In [9]:
model_pred = model.predict(test_data, batch_size=10,
                           callbacks=[LossAndMetricCallback()], verbose=False)

Finished prediction on batch 0!
Finished prediction on batch 1!
Finished prediction on batch 2!
Finished prediction on batch 3!
Finished prediction on batch 4!


In [10]:
lr_schedule = [(4, 0.003), (7, 0.02), (11, 0.005), (15, 0.007)]

def get_new_epoch_lr(epoch, lr):
    
    epoch_in_schedule = [i for i in range(len(lr_schedule)) if lr_schedule[i][0] == int(epoch)]
    if len(epoch_in_schedule) > 0:
        return lr_schedule[epoch_in_schedule[0]][1]
    
    else:
        return lr

In [11]:
class LRScheduler(tf.keras.callbacks.Callback):
    def __init__(self, new_lr):
        super.(LRScheduler, self).__init__()
        self.new_lr = new_lr
        
    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, 'lr'):
            raise ValueError("Error: Optimizer does not have a learning rate")
            
        curr_rate = float(tf.keras.backend.get_value(self.model.optimizer.lr))
        scheduled_rate = self.new_lr(epoch, curr_rate)
        
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_rate)
        print("Learning rate for epoch {} is {:.3f}".format(epoch, scheduled_rate))

SyntaxError: invalid syntax (<ipython-input-11-167878ef26ce>, line 3)